In [1]:
from shapely.geometry import Point
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio
import dask.dataframe as dd
import fastparquet
from shapely.ops import nearest_points

In [2]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

#finish indicator
wow = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = wow, autoplay = True) 

In [4]:
taxi_zone = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp')#.to_crs('epsg:4326')
taxi_zone = taxi_zone.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

zip_code = gpd.read_file('../../Data/ZIP CODES SHAPE/ZIP_CODE_040114.shp')#.to_crs('epsg:4326')
gpd.sjoin(gpd.GeoDataFrame(crs='epsg:4326',
    geometry=[Point(-73.966, 40.78)]), 
    zip_code, how='left', op='within')

complete

In [5]:
zip_code = gpd.read_file('../../Data/ZIP CODES SHAPE/ZIP_CODE_040114.shp')#.to_crs('epsg:4326')

complete

In [6]:
zip_code

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN,geometry
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0,"POLYGON ((1038098.252 188138.380, 1038141.936 ..."
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1001613.713 186926.440, 1002314.243 ..."
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1011174.276 183696.338, 1011373.584 ..."
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995908.365 183617.613, 996522.848 18..."
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((991997.113 176307.496, 992042.798 17..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,10310,0,Staten Island,25003.0,5.346328e+07,NY,Richmond,36,085,http://www.usps.com/,0.0,0.0,"POLYGON ((950767.507 172848.969, 950787.510 17..."
259,11693,0,Far Rockaway,11052.0,3.497516e+06,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((1028453.995 167153.410, 1027813.010 ..."
260,11249,0,Brooklyn,28481.0,1.777221e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0,"POLYGON ((995877.318 203206.075, 995968.511 20..."
261,10162,1,New York,0.0,2.103489e+04,NY,New York,36,061,http://www.usps.com/,0.0,0.0,"POLYGON ((997731.761 219560.922, 997641.948 21..."


In [7]:
zip_points = zip_code.copy()
zip_points['geometry'] = zip_points['geometry'].centroid

In [8]:
taxi_points = taxi_zone.copy()
taxi_points['geometry'] = taxi_points['geometry'].centroid

In [9]:
taxi_points=taxi_points.to_crs('epsg:4326')
zip_points=zip_points.to_crs('epsg:4326')

In [10]:
taxi_points

,zone,LocationID,borough,geometry
0,Newark Airport,1,EWR,POINT (-74.17400 40.69183)
1,Jamaica Bay,2,Queens,POINT (-73.83130 40.61675)
2,Allerton/Pelham Gardens,3,Bronx,POINT (-73.84742 40.86447)
3,Alphabet City,4,Manhattan,POINT (-73.97697 40.72375)
4,Arden Heights,5,Staten Island,POINT (-74.18849 40.55266)
...,...,...,...,...
258,Woodlawn/Wakefield,259,Bronx,POINT (-73.85222 40.89793)
259,Woodside,260,Queens,POINT (-73.90631 40.74423)
260,World Trade Center,261,Manhattan,POINT (-74.01302 40.70914)
261,Yorkville East,262,Manhattan,POINT (-73.94651 40.77593)


In [11]:
taxi_points['lat_lng']=taxi_points['geometry']

In [12]:
zip_points['lat_lng']=zip_points['geometry']

In [13]:
taxi_points['lat_lng']=taxi_points['lat_lng'].astype(str)

In [14]:
zip_points['lat_lng']=zip_points['lat_lng'].astype(str)

In [15]:
taxi_points_split=taxi_points['lat_lng'].str.strip('POINT(') \
                      .str.strip('(') \
                      .str.strip(')') \
                      .str.split(' ', expand=True)                   \
                     .rename(columns={1:'lat', 2:'lon'}) 

In [16]:
zip_points_split=zip_points['lat_lng'].str.strip('POINT(') \
                      .str.strip('(') \
                      .str.strip(')') \
                      .str.split(' ', expand=True)                   \
                     .rename(columns={1:'lat', 2:'lon'}) 

In [ ]:
taxi_points=taxi_points.join(taxi_points_split)

In [ ]:
zip_points=zip_points.join(taxi_points_split)

In [ ]:
taxi_points['Llat']=taxi_points['lat'].str.strip('(')

In [ ]:
zip_points['lat']=zip_points['lat'].str.strip('(')

In [ ]:
taxi_points=taxi_points[['zone','LocationID','borough','geometry','lat','lon']]

In [ ]:
zip_points=zip_points[['ZIPCODE','PO_NAME','COUNTY','geometry','lat','lon']]

In [ ]:
taxi_points

In [ ]:
zip_points.info()

taxi_points=taxi_points.rename(columns = {'Latitude':'lat','Longitude':'lon'})

zip_points=zip_points.rename(columns = {'Latitude':'lat','Longitude':'lon'})

In [ ]:
def find_nearest(lat, long):
    distances = zip_points.apply(
        lambda row: dist(lat, long, row['lat'], row['lon']), 
        axis=1)
    return zip_points.loc[distances.idxmin(), 'ZIPCODE']

In [ ]:
taxi_points

In [ ]:
taxi_points[['lat','lon']]=taxi_points[['lat','lon']].astype(float)

In [ ]:
zip_points[['lat','lon']]=zip_points[['lat','lon']].astype(float)

In [ ]:
zip_points.info()

In [ ]:
taxi_points['closest'] = taxi_points.apply(
    lambda row: find_nearest(row['lat'], row['lon']), 
    axis=1)
# To check the data frame if it has a new column of hotel name (for each and every member's location in the list)
taxi_points.head()

In [ ]:
def assign_taxi_zones(df, lon_var, lat_var, locid_var):

    localdf = df[[lon_var, lat_var]].copy()
    localdf[lon_var] = localdf[lon_var].fillna(value=0.)
    localdf[lat_var] = localdf[lat_var].fillna(value=0.)
    

    shape_df = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp')
    shape_df.drop(['OBJECTID', "Shape_Area", "Shape_Leng", "borough", "zone"],
                  axis=1, inplace=True)
    shape_df = shape_df.to_crs('EPSG:4326')

    try:
        local_gdf = gpd.GeoDataFrame(
            localdf, crs=('EPSG:4326'),
            geometry=[Point(xy) for xy in
                      zip(localdf[lon_var], localdf[lat_var])])

        local_gdf = gpd.sjoin(
            local_gdf, shape_df, how='left', op='within')

        return local_gdf.LocationID.rename(locid_var)
    except ValueError as ve:
        print(ve)
        print(ve.stacktrace())
        series = localdf[lon_var]
        series = np.nan
        return series

In [ ]:
trips= dd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-01.csv',dtype={' tolls_amount': 'float64'})

In [ ]:
trips['pickup_taxizone_id'] = trips.map_partitions(
    assign_taxi_zones, " pickup_longitude", " pickup_latitude",
    "pickup_taxizone_id", meta=('pickup_taxizone_id', np.float64))
trips['dropoff_taxizone_id'] = trips.map_partitions(
    assign_taxi_zones, " dropoff_longitude", " dropoff_latitude",
    "dropoff_taxizone_id", meta=('dropoff_taxizone_id', np.float64))
trips[['pickup_taxizone_id', 'dropoff_taxizone_id']].head()

#assume_missing=True

In [ ]:
start_time = time.time()

df.to_parquet('df_2014-01.parquet', has_nulls=True,
     object_encoding='json', compression="SNAPPY")
df = dd.read_parquet('df_2014-015.parquet',
    columns=['pickup_taxizone_id', 'dropoff_taxizone_id'])

print("--- %s seconds ---" % (time.time() - start_time))

complete

In [ ]:
trips.info()

In [ ]:
trips.describe()

In [ ]:
trips2= dd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-01.csv',dtype={' tolls_amount': 'float64'})

In [ ]:
trips2.tail()

In [ ]:
trips.tail()

In [ ]:
trips3=trips2.merge(trips2,trips, how=index)

In [ ]:
trips4= pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-01.csv',dtype={' tolls_amount': 'float64'})

In [ ]:
trips3=trips2.merge(trips, how='inner', on=None)

In [ ]:
trips3.head()

In [ ]:
trips4.info()